In [104]:
%matplotlib ipympl
import pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytransform3d.rotations import *

In [105]:
# method to map from image plane to estimated groud plane (GP)
# 1) for each KF, select visible points from map_points
# 2) get four map_points from subset with smallest distance from GP
# 3) project these points onto the GP
# 4) use cv2.getPerspectiveTransform() to compute a homography between the four selected and projected GP points and their correpsonding image points in the KF
# 5) project KF image onto GP by using cv2.warpPerspective()

# alternatives:
# 2b) get N>4 (e.g. N=10) map_points with smallest distance from GP
# 3b) project points onto GP
# 4b) use cv2.findHomography() to estimate the best fit homography via RANSAC

In [106]:
map_points = pickle.load(open("../map_points.pkl", "rb"))
kf_visible_map_points = pickle.load(open("../kf_visible_map_points.pkl", "rb"))
kf_poses = pickle.load(open("../kf_poses.pkl", "rb"))
kf_frames = pickle.load(open("../kf_frames.pkl", "rb"))
kf_kp_matched = pickle.load(open("../kf_kp_matched.pkl", "rb"))

### Fit a Plane into the map points using a RANSAC scheme

In [107]:
import numpy as np
import numpy.linalg as la

eps = 0.00001

def svd(A):
    u, s, vh = la.svd(A)
    S = np.zeros(A.shape)
    S[:s.shape[0], :s.shape[0]] = np.diag(s)
    return u, S, vh


def fit_plane_LSE(points):
    # points: Nx4 homogeneous 3d points
    # return: 1d array of four elements [a, b, c, d] of
    # ax+by+cz+d = 0
    assert points.shape[0] >= 3 # at least 3 points needed
    U, S, Vt = svd(points)
    null_space = Vt[-1, :]
    return null_space


def get_point_dist(points, plane):
    # return: 1d array of size N (number of points)
    dists = np.abs(points @ plane) / np.sqrt(plane[0]**2 + plane[1]**2 + plane[2]**2)
    return dists


def fit_plane_LSE_RANSAC(points, iters=1000, inlier_thresh=0.05, num_support_points=None, return_outlier_list=False):
    # points: Nx4 homogeneous 3d points
    # num_support_points: If None perform LSE fit with all points, else pick `num_support_points` random points for fitting
    # return: 
    #   plane: 1d array of four elements [a, b, c, d] of ax+by+cz+d = 0
    #   inlier_list: 1d array of size N of inlier points
    max_inlier_num = -1
    max_inlier_list = None
    
    N = points.shape[0]
    assert N >= 3

    for i in range(iters):
        chose_id = np.random.choice(N, 3, replace=False)
        chose_points = points[chose_id, :]
        tmp_plane = fit_plane_LSE(chose_points)
        
        dists = get_point_dist(points, tmp_plane)
        tmp_inlier_list = np.where(dists < inlier_thresh)[0]
        tmp_inliers = points[tmp_inlier_list, :]
        num_inliers = tmp_inliers.shape[0]
        if num_inliers > max_inlier_num:
            max_inlier_num = num_inliers
            max_inlier_list = tmp_inlier_list
        
        #print('iter %d, %d inliers' % (i, max_inlier_num))

    final_points = points[max_inlier_list, :]
    if num_support_points:
        max_support_points = np.min((num_support_points, final_points.shape[0]))
        support_idx = np.random.choice(np.arange(0, final_points.shape[0]), max_support_points, replace=False)
        support_points = final_points[support_idx, :]
    else:
        support_points = final_points
    print(final_points.shape)
    plane = fit_plane_LSE(support_points)
    
    fit_variance = np.var(get_point_dist(final_points, plane))
    print('RANSAC fit variance: %f' % fit_variance)
    print(plane)

    dists = get_point_dist(points, plane)

    select_thresh = inlier_thresh * 1

    inlier_list = np.where(dists < select_thresh)[0]
    if not return_outlier_list:
        return plane, inlier_list
    else:
        outlier_list = np.where(dists >= select_thresh)[0]
        return plane, inlier_list, outlier_list

In [108]:
map_points_h = cv2.convertPointsToHomogeneous(map_points).reshape(-1, 4)

In [109]:
import time

In [110]:
t0 = time.perf_counter()
plane, inlier_list = fit_plane_LSE_RANSAC(map_points_h, iters=1000, inlier_thresh=1, num_support_points=3000, return_outlier_list=False)
dt = time.perf_counter() - t0
print(inlier_list.shape, dt)

(29862, 4)
RANSAC fit variance: 0.047503
[-1.19081787e-04  3.21561617e-04 -5.06192658e-02  9.98717964e-01]
(29467,) 1.2476416120771319


### Project map points onto plane along the plane normal

In [111]:
# project all map points onto the plane
def plane_to_hessian(plane):
    """Convert plane to Hessian normal form (n.x + p = 0)"""
    a, b, c, d = plane
    nn = np.sqrt(a**2+b**2+c**2)
    n = np.array([a/nn, b/nn, c/nn])
    p = d/nn
    return n, p

def project_points(plane, points):
    """Project points with shape (-1, 3) onto plane (given as coefficients a, b, c, d with ax+by+cz+d=0)."""
    n, p = plane_to_hessian(plane)
    p_orig = -n*p
    points_proj = points.reshape(-1, 3) - (np.sum(n*(points.reshape(-1, 3) - p_orig.reshape(1, 3)), axis=1)*n.reshape(3, 1)).T
    return points_proj

In [112]:
map_points_proj = project_points(plane, map_points[inlier_list])

In [113]:
map_points_proj

array([[ -8.23596354,  -0.72518199,  19.7447654 ],
       [-15.15439958,  -3.60200864,  19.74276582],
       [  4.93940285,   7.37964864,  19.76525673],
       ...,
       [ 18.51001446,   1.46639143,  19.69576759],
       [ -0.57625531,  -8.83460476,  19.67523039],
       [ 10.92487029,   2.40606333,  19.71958095]])

### Transform map points from world coordinates to plane coordinates

Choose a random orthonormal base inside the plane. Find an affine transformation M that maps 3D world coordinates of the projected map points (projected onto the plane) to 2D plane coordinates. These plane coordinates are defined with respect to the orthonormal base. Based on this stackoverflow question: https://stackoverflow.com/a/18522281

In [114]:
# transform points from world to plane coordinates
def get_world2plane_transformation(plane, points, return_base=True):
    """Yields an affine transformation M from 3D world to 2D plane coordinates.

    Args:
    
        plane (`numpy.ndarray`): Shape (4,). Plane coefficients [a, b, c, d] which fullfill
            ax + by + cz + d = 0.

        points (`numpy.ndarray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.
        
        return_base (`bool`): Wether to return the orthonormal base or not.
    
    Returns:
    
        M (`numpy.ndarray`): Shape (4, 4). Affine transformation matrix which maps points on the plane
        from 3D world coordinates to 2D points with respect to a randomly chosen orthonormal base on the plane.
        Compute point2D = M @ point3D to map a 3D point to 2D coordinates. To retrieve a 3D point from 2D 
        planes coordinates compute point3D = inv(M) @ point2D.
        
        base (`numpy.ndarray`) Shape (3, 4). right-handend orthonormal base in which 2D plane points are 
        expressed. Column vectors of the array correspond to the origin point O of the base, the first and 
        second base vector u, v and the third base vector n which is the plane normal.  
    """
    # pick a random point on the plane as origin and another one to form first base vector
    point_idx = np.arange(0, points.shape[0])
    np.random.seed(0)
    plane_O, plane_U = points[np.random.choice(point_idx, 2, replace=False), :]
    u = (plane_U - plane_O)/np.linalg.norm(plane_U - plane_O)
    n, _ = plane_to_hessian(plane)  # plane normal
    # compute third base vector
    v = np.cross(u, n)/np.linalg.norm(np.cross(u, n))
    # get end points of base vectors
    U = plane_O + u
    V = plane_O + v
    N = plane_O + n
    # form base quadruplet
    D = np.array([[0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1],
                  [1, 1, 1, 1]])
    # form transformation matrix S with M * S = D
    S = np.stack((np.append(plane_O, 1), np.append(U, 1), np.append(V, 1), np.append(N, 1)), axis=1)
    # compute affine transformation M which maps points from world to plane coordinates
    M = np.matmul(D, np.linalg.inv(S))
    if return_base:
        base = np.stack([plane_O.T, u.T, v.T, n.T], axis=1)
        return M, base
    else:
        return M


def map_points_world2plane(points_world, M):
    """Transforms 3D points on a plane to 2D plane coordinates given the transformation matrix M.
    
    Args:
    
        points_world (`numpy.ndarrray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.

        M (`nnumpy.ndarray`): Shape (4, 4). Affine transformation matrix computed with `get_world2plane_transformation`.
    
    Returns:
    
        points_plane (`numpy.ndarrray`): Shape (-1, 2). 2D points on the plane in (xp, yp, z=0) plane coordinates 
        w.r.t. to a randomly chosen orthonormal base.
    """
    points_world_h = cv2.convertPointsToHomogeneous(points_world).reshape(-1, 4)
    points_plane_h = (M @ points_world_h.T).T
    points_plane = cv2.convertPointsFromHomogeneous(points_plane_h).reshape(-1, 3)
    points_plane = points_plane[:, :2]
    return points_plane


def map_points_plane2world(points_plane, M):
    """Transforms 2D plane points into 3D world coordinates given the transformation matrix M.
    
    Args:
    
        points_plane (`numpy.ndarrray`): Shape (-1, 2). 2D points on the plane in (xp, yp, z=0) plane coordinates 
        w.r.t. to a randomly chosen orthonormal base.

        M (`nnumpy.ndarray`) Shape (4, 4). Affine transformation matrix computed with `get_world2plane_transformation`.
    
    Returns:
    
        points_world (`numpy.ndarrray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.
    """
    points_plane_tmp = np.zeros((points_plane.shape[0], 3))
    points_plane_tmp[:, :2] = points_plane
    points_plane_h = cv2.convertPointsToHomogeneous(points_plane_tmp).reshape(-1, 4)
    points_world_h = (np.linalg.inv(M) @ points_plane_h.T).T
    points_world = cv2.convertPointsFromHomogeneous(points_world_h).reshape(-1, 3)
    return points_world

In [115]:
# test functions
points_world = map_points_proj
print(points_world)
M, base = get_world2plane_transformation(plane, points_world)
points_plane = map_points_world2plane(points_world, M)
print(points_plane)
#points_world = map_points_plane2world(points_plane, M)  # yields the original points which shows the mapping is correct
#print(points_world)

[[ -8.23596354  -0.72518199  19.7447654 ]
 [-15.15439958  -3.60200864  19.74276582]
 [  4.93940285   7.37964864  19.76525673]
 ...
 [ 18.51001446   1.46639143  19.69576759]
 [ -0.57625531  -8.83460476  19.67523039]
 [ 10.92487029   2.40606333  19.71958095]]
[[20.51738853  2.53198484]
 [23.2517449  -4.44398735]
 [12.68437501 15.87075833]
 ...
 [18.87489461 29.31733434]
 [28.78240108 10.02386514]
 [17.77980062 21.75302792]]


In [116]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_world[:1000, 0], points_world[:1000, 1], points_world[:1000, 2], s=1, c="red")
ax.scatter(points_plane[:1000, 0], points_plane[:1000, 1], s=1, c="green")
ax.set_xlim([-20,20])
ax.set_ylim([-20,20])
ax.set_zlim([0,40])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_aspect(1.0)

# plot base points (point_O, U, V, N)
ax.scatter(base[0, 0], base[1, 0], base[2, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], base[2, 0]+base[2, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], base[2, 0]+base[2, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], base[2, 0]+base[2, 3], c="blue")

ax.scatter(0, 0, 0, c="black", s=5)
plt.show() 

FigureCanvasNbAgg()

In [117]:
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
ax.scatter(points_world[:1000, 0], points_world[:1000, 1], s=1, c="red")
ax.scatter(points_plane[:1000, 0], points_plane[:1000, 1], s=1, c="green")
ax.set_xlim([-30,20])
ax.set_ylim([-20,20])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_aspect(1.0)
ax.scatter(base[0, 0], base[1, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], c="blue")
ax.scatter(0, 0, c="black", s=5)

# base [[Ox, ux, vx, nx],
#       [Oy, uy, vy, ny],
#       [Oz, uz, vz, nz]]

ax.grid()
plt.show() 

FigureCanvasNbAgg()

### Convert camera poses from world to plane coordinates

In [394]:
def to_twist(R, t):
    """Convert a 3x3 rotation matrix and translation vector (shape (3,))
    into a 6D twist coordinate (shape (6,))."""
    r, _ = cv2.Rodrigues(R)
    twist = np.zeros((6,))
    twist[:3] = r.reshape(3,)
    twist[3:] = t.reshape(3,)
    return twist

def from_twist(twist):
    """Convert a 6D twist coordinate (shape (6,)) into a 3x3 rotation matrix
    and translation vector (shape (3,))."""
    r = twist[:3].reshape(3, 1)
    t = twist[3:].reshape(3, 1)
    R, _ = cv2.Rodrigues(r)
    return R, t

In [395]:
# first compute R_plane, t_plane from given coordinate systems
# see: https://math.stackexchange.com/questions/1125203/finding-rotation-axis-and-angle-to-align-two-3d-vector-bases
def get_plane_pose(plane_O, plane_u, plane_v, plane_n):
    """Compute the plane pose R_plane, t_plane in world coordinates.
    
    Assumes the world coordinate system to have rotation R = I and zero translation.
    
    Args:
        plane_O (`numpy.ndarray`): Shape (3,). Origin of the plane coordinate base.
        
        plane_u (`numpy.ndarray`): Shape (3,). First base vector of the plane coordinate system.
        
        plane_v (`numpy.ndarray`): Shape (3,). Second base vector of the plane coordinate system.
        
        plane_n (`numpy.ndarray`): Shape (3,). Third base vector of the plane coordinate system, 
            corresponds to plane normal.
            
    Returns:
        R_plane (`numpy.ndarray`), t_plane (`numpy.ndarray`): Rotation matrix with shape (3, 3) and 
        translation vector with shape (3,) which describe the pose of the plane coordinate base in
        the world coordinate frame.
    """
    t_plane = plane_O
    world_x = np.array([1, 0, 0])
    world_y = np.array([0, 1, 0])
    world_z = np.array([0, 0, 1])
    R_plane = np.outer(plane_u, world_x) + np.outer(plane_v, world_y) + np.outer(plane_n, world_z)
    return R_plane, t_plane

In [396]:
R_plane, t_plane = get_plane_pose(base[:, 0], base[:, 1], base[:, 2], base[:, 3])
print(R_plane, t_plane)

[[ 0.0204988   0.99978711 -0.00235245]
 [-0.9997694   0.02051339  0.00635241]
 [-0.00639931 -0.00222169 -0.99997706]] [-11.18799128  19.73553558  19.88168785]


In [307]:
def invert_pose(R, t):
    """Inverts a pose described by 3x3 rotation matrix R and 3d-vector t."""
    R_inv = R.T
    t_inv = -np.matmul(R.T, t.reshape(3,))
    return R_inv, t_inv

In [308]:
def pose_world_to_plane(R_plane, t_plane, R_pose, t_pose):
    """"""
    t_plane = t_plane.reshape(3,)
    t_pose = t_pose.reshape(3,)
    # invert mapping to retrieve a mapping from world to plane coordinates
    R_plane_inv, t_plane_inv = invert_pose(R_plane, t_plane)
    # now map camera pose from world to plane coordinates
    R_pose_plane = R_plane_inv @ R_pose
    t_pose_plane = t_pose.reshape(3,) + R_pose @ t_plane_inv
    return R_pose_plane, t_pose_plane

In [397]:
def pose_world_to_plane(R_plane, t_plane, R_pose, t_pose):
    """Map keyframe poses from the world to plane coordinate frame.

    Args:
        R_plane (`numpy.ndarray`): Shape (3, 3). Rotation matrix of plane coordinate system in world coordinate frame.
        t_plane (`numpy.ndarray`): Shape (3,). Translation vector of plane coordinate system in world coordinate frame.
        
        R_pose (`numpy.ndarray`): Shape (3, 3). Rotation matrix of keyframe in world coordinate frame.
        t_pose (`numpy.ndarray`): Shape (3,). Translation vector of keyframe in world coordinate frame.

    Returns:
        R_pose_plane (`numpy.ndarray`), t_pose_plane (`numpy.ndarray`): Rotation matrix with shape (3, 3) and 
        translation vector with shape (3,) of the keyframe in plane coordinate frame.
    """
    t_pose_plane = t_plane.reshape(3,) + R_plane @ t_pose.reshape(3,)
    R_pose_plane = R_plane @ R_pose
    return R_pose_plane, t_pose_plane

In [398]:
# convert all keyframe poses to plane coordinates
kf_poses_plane = []
for pose in kf_poses:
    R_pose, t_pose = from_twist(pose)
    R_kf_plane, t_kf_plane = pose_world_to_plane(R_plane, t_plane, R_pose, t_pose)
    kf_poses_plane.append((R_kf_plane, t_kf_plane))

In [399]:
R_world = np.eye(3)
t_world = np.zeros((3,))

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
plot_basis(ax, R_world, t_world)
plot_basis(ax, R_plane, t_plane)

R_0, t_0 = from_twist(kf_poses[0])
plot_basis(ax, R_0, t_0.reshape(3,))
R_1, t_1 = from_twist(kf_poses[1])
plot_basis(ax, R_1, t_1.reshape(3,))

R_0, t_0 = kf_poses_plane[0]
plot_basis(ax, R_0, t_0.reshape(3,))
R_1, t_1 = kf_poses_plane[1]
plot_basis(ax, R_1, t_1.reshape(3,))

ax.scatter(base[0, 0], base[1, 0], base[2, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], base[2, 0]+base[2, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], base[2, 0]+base[2, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], base[2, 0]+base[2, 3], c="blue")
ax.set_xlim([-20, 20])
ax.set_ylim([-20, 40])
ax.set_zlim([0, 40])
ax.set_aspect(1.0)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

### Project image corners onto plane

In [400]:
w = 1920
h = 1080
fx = 1184.51770
fy = 1183.63810
cx = 978.30778
cy = 533.85598

In [401]:
# corners of the image
img_points = np.array([[0, 0],
                       [w, 0],
                       [0, h],
                       [w, h]])

In [402]:
def unproject(img_points, fx, fy, cx, cy):
    """Unproject image points with shape (-1, 2) to camera coordinates."""
    camera_points = np.zeros((img_points.shape[0], 3))
    camera_points[:, 0] = (img_points[:, 0]-cx)/fx
    camera_points[:, 1] = (img_points[:, 1]-cy)/fy
    camera_points[:, 2] = 1.0
    return camera_points

In [403]:
camera_points = unproject(img_points, fx, fy, cx, cy)
camera_points  # image corners in camera coordinates

array([[-0.82591234, -0.45102974,  1.        ],
       [ 0.79500055, -0.45102974,  1.        ],
       [-0.82591234,  0.46141132,  1.        ],
       [ 0.79500055,  0.46141132,  1.        ]])

In [514]:
warped_frames = []
length_pixel = 0.01

for pose, frame in zip(kf_poses_plane, kf_frames):
    
    frame_okay = True
    cur_view_min = np.array([1e6, 1e6])
    cur_view_max = np.array([-1e6, -1e6])
    
    if pose[1][-1] < 0:
        downlook = np.array([0,0,1])
    else:
        downlook = np.array([0,0,-1])
    
    # project image corners from camera to plane coordinates
    plane_points = np.zeros((camera_points.shape[0], 2))
    for i, camera_point in enumerate(camera_points):
        axis = pose[0] @ camera_point
        if np.dot(axis, downlook) < 0.4:
            print("Camera axis is deviating too much from 'down' direction. Skipping to next keyframe.")
            frame_okay = False
            break
        axis = pose[1] - axis*(pose[1][-1]/axis[-1])
        plane_points[i, :] = axis[:2]
        
    if not frame_okay:
        continue
    
    # expand viewport if necessary
    for i, plane_point in enumerate(plane_points):
        if plane_point[0] < cur_view_min[0]:
            cur_view_min[0] = plane_point[0]
        if plane_point[1] < cur_view_min[1]:
            cur_view_min[1] = plane_point[1]
        if plane_point[0] > cur_view_max[0]:
            cur_view_max[0] = plane_point[0]
        if plane_point[1] > cur_view_max[1]:
            cur_view_max[1] = plane_point[1]

    size = ((cur_view_max - cur_view_min)/length_pixel).astype(np.int)    
    dst_points = (plane_points - cur_view_min)/length_pixel
    
    # find homography between camera and ground plane points
    transmtx = cv2.getPerspectiveTransform(img_points.astype(np.float32), dst_points.astype(np.float32))
    
    # warp image
    warped_frame = cv2.warpPerspective(frame, transmtx, tuple(size), cv2.INTER_CUBIC, cv2.BORDER_REFLECT)
    warped_frames.append(warped_frame)

In [515]:
fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111)
#alpha = 0.5
#dst = cv2.addWeighted(warped_frames[0], alpha, warped_frames[1], 1-alpha, 0.0)
#ax.imshow(dst)
ax.imshow(warped_frames[12])
ax.grid()
plt.show()

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [334]:
R_world = np.eye(3)
t_world = np.zeros((3,))

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
plot_basis(ax, R_world, t_world)
plot_basis(ax, R_plane, t_plane)

R_0, t_0 = from_twist(kf_poses[0])
plot_basis(ax, R_0, t_0.reshape(3,))
R_1, t_1 = from_twist(kf_poses[1])
plot_basis(ax, R_1, t_1.reshape(3,))

R_0, t_0 = kf_poses_plane[0]
plot_basis(ax, R_0, t_0.reshape(3,))
R_1, t_1 = kf_poses_plane[1]
plot_basis(ax, R_1, t_1.reshape(3,))

ax.scatter(points_world[:1000, 0], points_world[:1000, 1], points_world[:1000, 2], s=1, c="red")
ax.scatter(points_plane[:1000, 0], points_plane[:1000, 1], s=1, c="green")

ax.scatter(camera_points[:, 0], camera_points[:, 1], camera_points[:, 2], c="cyan")
ax.scatter(plane_points_0[:, 0], plane_points_0[:, 1], c="magenta")
ax.scatter(plane_points[:, 0], plane_points[:, 1], c="magenta")

ax.scatter(base[0, 0], base[1, 0], base[2, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], base[2, 0]+base[2, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], base[2, 0]+base[2, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], base[2, 0]+base[2, 3], c="blue")
ax.set_xlim([-20, 20])
ax.set_ylim([-20, 40])
ax.set_zlim([0, 40])
ax.set_aspect(1.0)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

### Find homography between camera plane and ground plane for each keyframe